# Library Load

In [1]:
import pandas as pd
import numpy as np
import random
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

from konlpy.tag import Mecab
from soynlp.hangle import compose, decompose, character_is_korean
from gensim.models import FastText, Word2Vec
from sklearn.feature_extraction.text import CountVectorizer
import itertools
import logging
import re

from collections import Counter
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import time

import lightgbm as lgb
from lightgbm import plot_importance
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, f1_score

import warnings
warnings.filterwarnings('ignore')

C:\Users\user\.conda\envs\green\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
version = 'v2'

# Data Load

In [3]:
train = pd.read_csv('./data/train.csv')

In [4]:
train

,index,제출년도,사업명,사업_부처명,계속과제여부,내역사업명,과제명,요약문_연구목표,요약문_연구내용,요약문_기대효과,요약문_한글키워드,요약문_영문키워드,label
0,0,2016,농업기초기반연구,농촌진흥청,신규,농산물안전성연구,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,○ 새로운 해충분류군의 동정기술 개발 및 유입확산 추적,(가) 외래 및 돌발해충의 발생조사 및 종 동정\n\n\n ○ 대상해충 : 최...,"○ 새로운 돌발 및 외래해충의 신속, 정확한 동정법 향상\n\n\n○ 돌발 및 외래...","뉴클레오티드 염기서열, 분자마커, 종 동정, 침샘, 전사체","nucleotide sequence, molecular marker, species...",24
1,1,2019,이공학학술연구기반구축(R&D),교육부,신규,지역대학우수과학자지원사업(1년~5년),대장암의 TRAIL 내성 표적 인자 발굴 및 TRAIL 반응 예측 유전자 지도 구축...,최종목표: TRAIL 감수성 표적 유전자를 발굴하고 내성제어 기전을 연구. 발굴된...,1차년도\n1) Microarray를 통한 선천적 TRAIL 내성 표적 후보 유전자...,"1) TRAIL 내성 특이적 표적분자를 발굴하고, 이를 이용한 TRAIL 효과 증진...","대장암,항암제 내성,세포사멸,유전자발굴","TRAIL,Colorectal cancer,TRAIL resistance,Apopt...",0
2,2,2016,중소기업기술혁신개발,중소기업청,신규,혁신기업기술개발,비목질계 셀룰로오스 식물자원을 활용한 기능성 부직포 및 고부가가치 뷰티케어 faci...,* 식물계자원 정련 및 최적 신서란 파이버 기초연구 개발\n\n\n* 소비자 및 바...,* 식물계자원 정련 및 최적 신서란 파이버 기초연구 개발\n\n\n- Tencel/...,* 국내 독자적인 비목질계 셀룰로오스 자원의 파이버 및 부직포 제조 등의 기술 확보...,"기능성 샐룰로오스 파이버,천연섬유,기능성 부직포,뷰티&amp;케어 제품,미용 솜","functional cellulose fiber,natural fiber,funct...",0
3,3,2018,창업성장기술개발(R&D),중소벤처기업부,신규,창업사업화연계과제,소화기 암 진단용 분자영상 형광프로브 개발,# 암특이적 바이오마커 발굴 및 바이오마커에 대한 프로브 개발\n\n\n1) 소화...,# 소화기 암 진단용 분자영상 형광프로브 개발\n\n\n- 국소 도포형 소화기 암 ...,# 암 진단기술의 차별성: 소화기 암 특이 프로브 개발\n\n\n- 최근 체외진단시...,"분자 진단,형광 조영제,프로브,항체,대장암","Molecular diagnosis,Fluorescence,probe,antibod...",0
4,4,2016,이공학개인기초연구지원,교육부,계속,기본연구지원,위암환자의 항암제반응예측을 위한 BRCA 발현검사,"수술이 불가능한 위암환자는 생존기간은 10개월 안팎에 지나지 않고, 항암화학요법에 ...",-In situ hybridization 검사의 정확성을 확인하기 위해 위암세포주 ...,-본 연구는 파라핀보관조직에서 in situ hybridization로 mRNA 및...,"BRCA,제자리부합법,조직미세배열,마이크로RNA,위암,항암제반응,젊은 연령/가족성 위암","BRCA,Insituhybridization,tissuemicroarray,micr...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
174299,174299,2016,개인연구지원,미래창조과학부,신규,중견연구,혈관내막 증식을 치료하기 위한 표적분자 발굴,"증식성 혈관질환과 염증반응과의 관련성이 높으나, 혈관평활근세포의 탈분화를 유도하는 ...",● 1차년: 혈관내막 증식과 관련된 표적분자 발굴 및 염증세포의 활성과 분화에 대한...,심혈관 질환 발생의 새로운 기전 규명뿐만 아니라 제어할 수 있는 새로운 방법을 탐...,"혈관내막 증식,혈관 개조,혈관평활근세포,염증세포,혈관염증,혈관평활근세포 탈분화,표적분자","Intimahyperplasia,Vascularremodeling,Vasculars...",0
174300,174300,2016,개인연구지원,미래창조과학부,신규,중견연구,삼킴곤란 개선을 위한 물성조절 식품시스템에서의 유동층과립 복합소재의 유변물성 변화 ...,삼킴곤란을 겪고 있는 고령자의 안전한 식품섭취는 삼킴에 적합한 유변물성을 가진 물성...,본 연구에서는 여러 가지 유변물성 측정기술을 이용하여 고령자의 삼킴곤란 개선을 위해...,○ 그 동안 연구되지 않은 다양한 물성조절 검복합 소재의 과립제형 및 유변물성 연구...,"삼킴곤란,유변물성,유동층과립,물리적 특성,검복합물","Swallowingdifficulty,Rheologicalproperty,Fludi...",0
174301,174301,2018,이공학학술연구기반구축(R&D),교육부,계속,이공분야학문후속세대양성_대통령POST_DOC펠로우쉽,식물 유전체 정보 고도화 및 핵심 유전자 군 진화연구,"지난 10년여 동안 100여종에 대한 식물 유전체 정보가 완성되어 공개되었지만, 애...",○ 공개된 식물 유전체 정보 및 대상 유전자 군 선발\n\n - 공개된 식물 유...,- 본 과제에서 개발한 annotation tool은 유전체 고도화 부분에서 가장 ...,"유전자 정보 고도화,대상 유전자 군 재분석,비교 유전체 진화 분석,공개 식물 유전체...","Gene model improvement,Target gene family re-a...",0
174302,174302,2019,수요자맞춤형육종자원대량신속발굴기술개발(R&D),농촌진흥청,신규,유용형질 대량 특성평가를 통한 육종모본 발굴,"콩 유전자원 내건성, 내습성 대량평가 체계구축 및 자원선발",1) 내건 및 내습성 유전자원 수집\n\n2) 내건성 자원평가 체계구축\n\n3) ...,[1차 년도(2019)] \n\n1) 내건 및 내습성 유전자원 수집\n\n○ 기존 ...,○ 내건성 및 내습성 특성평가 체계구축 및 우수자원 발굴로 내재해성 콩 품종육성 자...,"콩, 가공적성, 병 저항성, 내재해성, 콩모자이크바이러스","soybean, food application, resistance, biotic-...",23


# Preprocessing

In [5]:
def text_preprocessing(text_list):
    
    stopwords = ['은','는','이','가'] 
    tokenizer = Mecab(dicpath=r'C:\mecab\mecab-ko-dic') 
    token_list = []
    
    for text in tqdm(text_list):
        #text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ]","", text)
        #text = re.sub('[^A-Za-z0-9가-힣]', '', text)
        text = re.sub('[^0-9가-힣]', '', text)
        #text = text.lower()
        token = tokenizer.morphs(text) 
        token = [t for t in token if t not in stopwords or type(t) != float]
        token_list.append(token)
        
    return token_list

In [6]:
train.isnull().sum()

index           0
제출년도            0
사업명             0
사업_부처명          0
계속과제여부          0
내역사업명           0
과제명             0
요약문_연구목표     3002
요약문_연구내용     3001
요약문_기대효과     3051
요약문_한글키워드    3028
요약문_영문키워드    3087
label           0
dtype: int64

In [7]:
train = train.fillna('NAN')
train.isnull().sum()

index        0
제출년도         0
사업명          0
사업_부처명       0
계속과제여부       0
내역사업명        0
과제명          0
요약문_연구목표     0
요약문_연구내용     0
요약문_기대효과     0
요약문_한글키워드    0
요약문_영문키워드    0
label        0
dtype: int64

In [8]:
train_dum = pd.get_dummies(train, columns = ['제출년도', '사업_부처명', '계속과제여부'])
train_dum.head()

,index,사업명,내역사업명,과제명,요약문_연구목표,요약문_연구내용,요약문_기대효과,요약문_한글키워드,요약문_영문키워드,label,...,사업_부처명_중소기업청,사업_부처명_중소벤처기업부,사업_부처명_특허청,사업_부처명_해양경찰청,사업_부처명_해양수산부,사업_부처명_행정안전부,사업_부처명_행정자치부,사업_부처명_환경부,계속과제여부_계속,계속과제여부_신규
0,0,농업기초기반연구,농산물안전성연구,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,○ 새로운 해충분류군의 동정기술 개발 및 유입확산 추적,(가) 외래 및 돌발해충의 발생조사 및 종 동정\n\n\n ○ 대상해충 : 최...,"○ 새로운 돌발 및 외래해충의 신속, 정확한 동정법 향상\n\n\n○ 돌발 및 외래...","뉴클레오티드 염기서열, 분자마커, 종 동정, 침샘, 전사체","nucleotide sequence, molecular marker, species...",24,...,0,0,0,0,0,0,0,0,0,1
1,1,이공학학술연구기반구축(R&D),지역대학우수과학자지원사업(1년~5년),대장암의 TRAIL 내성 표적 인자 발굴 및 TRAIL 반응 예측 유전자 지도 구축...,최종목표: TRAIL 감수성 표적 유전자를 발굴하고 내성제어 기전을 연구. 발굴된...,1차년도\n1) Microarray를 통한 선천적 TRAIL 내성 표적 후보 유전자...,"1) TRAIL 내성 특이적 표적분자를 발굴하고, 이를 이용한 TRAIL 효과 증진...","대장암,항암제 내성,세포사멸,유전자발굴","TRAIL,Colorectal cancer,TRAIL resistance,Apopt...",0,...,0,0,0,0,0,0,0,0,0,1
2,2,중소기업기술혁신개발,혁신기업기술개발,비목질계 셀룰로오스 식물자원을 활용한 기능성 부직포 및 고부가가치 뷰티케어 faci...,* 식물계자원 정련 및 최적 신서란 파이버 기초연구 개발\n\n\n* 소비자 및 바...,* 식물계자원 정련 및 최적 신서란 파이버 기초연구 개발\n\n\n- Tencel/...,* 국내 독자적인 비목질계 셀룰로오스 자원의 파이버 및 부직포 제조 등의 기술 확보...,"기능성 샐룰로오스 파이버,천연섬유,기능성 부직포,뷰티&amp;케어 제품,미용 솜","functional cellulose fiber,natural fiber,funct...",0,...,1,0,0,0,0,0,0,0,0,1
3,3,창업성장기술개발(R&D),창업사업화연계과제,소화기 암 진단용 분자영상 형광프로브 개발,# 암특이적 바이오마커 발굴 및 바이오마커에 대한 프로브 개발\n\n\n1) 소화...,# 소화기 암 진단용 분자영상 형광프로브 개발\n\n\n- 국소 도포형 소화기 암 ...,# 암 진단기술의 차별성: 소화기 암 특이 프로브 개발\n\n\n- 최근 체외진단시...,"분자 진단,형광 조영제,프로브,항체,대장암","Molecular diagnosis,Fluorescence,probe,antibod...",0,...,0,1,0,0,0,0,0,0,0,1
4,4,이공학개인기초연구지원,기본연구지원,위암환자의 항암제반응예측을 위한 BRCA 발현검사,"수술이 불가능한 위암환자는 생존기간은 10개월 안팎에 지나지 않고, 항암화학요법에 ...",-In situ hybridization 검사의 정확성을 확인하기 위해 위암세포주 ...,-본 연구는 파라핀보관조직에서 in situ hybridization로 mRNA 및...,"BRCA,제자리부합법,조직미세배열,마이크로RNA,위암,항암제반응,젊은 연령/가족성 위암","BRCA,Insituhybridization,tissuemicroarray,micr...",0,...,0,0,0,0,0,0,0,0,1,0


# CountVectorization

In [9]:
#busi_title_token = text_preprocessing(train['사업명'])
#busi_depart_token = text_preprocessing(train['사업_부처명'])
#busi_name_token = text_preprocessing(train['내역사업명'])
title_token = text_preprocessing(train['과제명'])
#goal_token = text_preprocessing(train['요약문_연구목표'])
#content_token = text_preprocessing(train['요약문_연구내용'])
#effect_token = text_preprocessing(train['요약문_기대효과'])
#ko_keyword_token = text_preprocessing(train['요약문_한글키워드'])
#en_keyword_token = text_preprocessing(train['요약문_영문키워드'])

100%|███████████████████████████████████████████████████████████████████████| 174304/174304 [00:06<00:00, 27129.26it/s]


In [10]:
len(title_token)

174304

In [11]:
vectorizer = CountVectorizer(tokenizer = lambda x: x, lowercase=False)

In [12]:
title_token_vec = vectorizer.fit_transform(title_token)

In [13]:
type(title_token_vec)

scipy.sparse.csr.csr_matrix

# Feature Selection

In [14]:
train_dum.columns.tolist()

['index',
 '사업명',
 '내역사업명',
 '과제명',
 '요약문_연구목표',
 '요약문_연구내용',
 '요약문_기대효과',
 '요약문_한글키워드',
 '요약문_영문키워드',
 'label',
 '제출년도_2016',
 '제출년도_2017',
 '제출년도_2018',
 '제출년도_2019',
 '사업_부처명_경찰청',
 '사업_부처명_과학기술정보통신부',
 '사업_부처명_교육부',
 '사업_부처명_국민안전처',
 '사업_부처명_국토교통부',
 '사업_부처명_기상청',
 '사업_부처명_기획재정부',
 '사업_부처명_농림축산식품부',
 '사업_부처명_농촌진흥청',
 '사업_부처명_다부처',
 '사업_부처명_문화재청',
 '사업_부처명_문화체육관광부',
 '사업_부처명_미래창조과학부',
 '사업_부처명_범부처 사업',
 '사업_부처명_보건복지부',
 '사업_부처명_산림청',
 '사업_부처명_산업통상자원부',
 '사업_부처명_소방청',
 '사업_부처명_식품의약품안전처',
 '사업_부처명_원자력안전위원회',
 '사업_부처명_중소기업청',
 '사업_부처명_중소벤처기업부',
 '사업_부처명_특허청',
 '사업_부처명_해양경찰청',
 '사업_부처명_해양수산부',
 '사업_부처명_행정안전부',
 '사업_부처명_행정자치부',
 '사업_부처명_환경부',
 '계속과제여부_계속',
 '계속과제여부_신규']

In [15]:
train_dum = train_dum[['제출년도_2016', '제출년도_2017', '제출년도_2018', '제출년도_2019',
                       '사업_부처명_경찰청', '사업_부처명_과학기술정보통신부', '사업_부처명_교육부', '사업_부처명_국민안전처',
                       '사업_부처명_국토교통부', '사업_부처명_기상청', '사업_부처명_기획재정부', '사업_부처명_농림축산식품부',
                       '사업_부처명_농촌진흥청', '사업_부처명_다부처', '사업_부처명_문화재청', '사업_부처명_문화체육관광부',
                       '사업_부처명_미래창조과학부', '사업_부처명_범부처 사업', '사업_부처명_보건복지부', '사업_부처명_산림청',
                       '사업_부처명_산업통상자원부', '사업_부처명_소방청', '사업_부처명_식품의약품안전처',
                       '사업_부처명_원자력안전위원회', '사업_부처명_중소기업청', '사업_부처명_중소벤처기업부', '사업_부처명_특허청',
                       '사업_부처명_해양경찰청', '사업_부처명_해양수산부', '사업_부처명_행정안전부', '사업_부처명_행정자치부',
                       '사업_부처명_환경부', '계속과제여부_계속', '계속과제여부_신규']]

In [16]:
train_dum.shape

(174304, 34)

In [17]:
train_dum

,제출년도_2016,제출년도_2017,제출년도_2018,제출년도_2019,사업_부처명_경찰청,사업_부처명_과학기술정보통신부,사업_부처명_교육부,사업_부처명_국민안전처,사업_부처명_국토교통부,사업_부처명_기상청,...,사업_부처명_중소기업청,사업_부처명_중소벤처기업부,사업_부처명_특허청,사업_부처명_해양경찰청,사업_부처명_해양수산부,사업_부처명_행정안전부,사업_부처명_행정자치부,사업_부처명_환경부,계속과제여부_계속,계속과제여부_신규
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
3,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
4,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174299,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
174300,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
174301,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
174302,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [18]:
import scipy.sparse as sp
from scipy.sparse import csr_matrix

train_dum_csr_mat = csr_matrix(train_dum)

In [19]:
type(train_dum_csr_mat)

scipy.sparse.csr.csr_matrix

In [20]:
X_train = sp.hstack((title_token_vec, train_dum_csr_mat))

In [21]:
X_train.shape

(174304, 36797)

In [22]:
Y_train = train['label']

In [23]:
Y_train.shape

(174304,)

# Training

In [24]:
train_x, val_x, train_y, val_y = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [25]:
forest = RandomForestClassifier(n_estimators=1000, random_state=42)

forest.fit(train_x, train_y)

RandomForestClassifier(n_estimators=1000, random_state=42)

In [26]:
pred = forest.predict(val_x)
pred

array([ 0,  0, 19, ...,  0,  0,  0], dtype=int64)

In [27]:
accuracy = accuracy_score(val_y, pred)
accuracy

0.9061702188692234

In [28]:
f1 = f1_score(val_y, pred, average='macro')
f1

0.660899604234287

# Inference

In [29]:
test = pd.read_csv('./data/test.csv')

In [30]:
test = test.fillna('NAN')

In [31]:
test_dum = pd.get_dummies(test, columns = ['제출년도', '사업_부처명', '계속과제여부'])
test_dum.head()

,index,사업명,내역사업명,과제명,요약문_연구목표,요약문_연구내용,요약문_기대효과,요약문_한글키워드,요약문_영문키워드,제출년도_2016,...,사업_부처명_중소기업청,사업_부처명_중소벤처기업부,사업_부처명_특허청,사업_부처명_해양경찰청,사업_부처명_해양수산부,사업_부처명_행정안전부,사업_부처명_행정자치부,사업_부처명_환경부,계속과제여부_계속,계속과제여부_신규
0,174304,경제협력권산업육성,자동차융합부품,R-FSSW 기술 적용 경량 차체 부품 개발 및 품질 평가를 위한 64채널 C-SC...,○ 차체 점용접부의 품질 검사를 위한 64채널 무선 기반 C-Scan 탐촉자 개발\...,○ 1차년도\n\n . 개발 탐촉 시스템의 성능 평가 위한 표준 시편 제작 시...,○ 기술적 파급효과\n\n - 본 연구에서 개발된 R-FSSW 접합 기술은 기존 ...,"마찰교반점용접, 비파괴 검사, 초음파 탐상, 씨 스캔, 용접 품질 평가","Friction Stir Spot Welding, Non-destructive ev...",1,...,0,0,0,0,0,0,0,0,0,1
1,174305,개인기초연구(과기정통부)(R&D),신진연구(총연구비5천이상~1.5억이하),다입자계를 묘사하는 편미분방정식에 대한 연구,자연계에는 입자의 개수가 아주 큰 다양한 다입자계가 존재한다. 이런 다입자계의 효...,연구과제1. 무한입자계의 동역학 / 작용소(operator) 방정식에 대한 연구\n...,본 연구는 물리학에서 중요한 대상인 다입자계를 묘사하는 모델방정식의 정당성을 보장하...,"다체계 방정식,동역학의 안정성,양자역학,고전역학,평균장 극한,고전극한,비상대론적 극한","many particle system,stability of dynamics,qua...",0,...,0,0,0,0,0,0,0,0,1,0
2,174306,개인기초연구(미래부),자유공모,저출생체중아 가족을 위한 지역사회중심의 퇴원후 추후관리프로그램 개발,"저출생체중아는 정상 체중 출생아에 비하여 운동, 정신, 행동 및 정서발달장애가 빈번...",본 연구는 퇴원 후 저출생체중아의 퇴원 후 추후관리 프로그램 중 가정방문 모델과 가...,본 연구는 저출생체중아 가족 대상의 지역사회기반의 추후관리 사업에 대한 구체적인 모...,"저출생체중아,추후관리,조기개입,자조모임,가정방문","lowbirthweightinfant,followup,earlyinterventio...",0,...,0,0,0,0,0,0,0,0,1,0
3,174307,산학연협력기술개발(R&D),전략 협력(지역유망),폴리싱용 5축 CNC 정밀 마이크로 시스템 개발,최근 초정밀 공작기계 시장의 트렌드를 살펴보면 다음과 같다.\n\n\n(1) 자동차...,(1) 0.5~1㎛의 가공정밀도(면조도) 구현\n\n\n - 국내에서는 연삭기를 제...,(1) 가공 공정 및 소요시간 단축\n\n\n 초정밀금형 가공시 정삭 후 추가적...,"초정밀가공,폴리싱 가공,마이크로,경면가공,난삭재가공","superprecision,hiprecision,micro,mirror like f...",0,...,0,1,0,0,0,0,0,0,0,1
4,174308,이공학학술연구기반구축(R&D),지역대학우수과학자지원사업(1년~5년),다결정재료의 미세조직구조를 고려한 확률론적 응력부식균열 생성예측모델 개발,본 연구의 최종목표는 다결정 금속재료의 미세조직구조를 고려한 확률론적 응력부식균열(...,"총 연구기간은 3년으로, 연차별 연구내용 및 범위는 다음과 같다.\n\n ∙ 1차...",본 연구의 중요성 및 활용가능성은 다음과 같다.\n\n ∙ 기존의 재료전체 ave...,"결정구조,응력부식균열,확률론적모델","Crystalline structure,Stress corrosion crackin...",0,...,0,0,0,0,0,0,0,0,1,0


In [32]:
#busi_title_token = text_preprocessing(test['사업명'])
#busi_depart_token = text_preprocessing(test['사업_부처명'])
#busi_name_token = text_preprocessing(test['내역사업명'])
title_token = text_preprocessing(test['과제명'])
#goal_token = text_preprocessing(test['요약문_연구목표'])
#content_token = text_preprocessing(test['요약문_연구내용'])
#effect_token = text_preprocessing(test['요약문_기대효과'])
#ko_keyword_token = text_preprocessing(test['요약문_한글키워드'])
#en_keyword_token = text_preprocessing(test['요약문_영문키워드'])

100%|█████████████████████████████████████████████████████████████████████████| 43576/43576 [00:03<00:00, 13589.56it/s]


In [33]:
test_title_token_vec = vectorizer.transform(title_token)

In [34]:
test_dum.columns.tolist()

['index',
 '사업명',
 '내역사업명',
 '과제명',
 '요약문_연구목표',
 '요약문_연구내용',
 '요약문_기대효과',
 '요약문_한글키워드',
 '요약문_영문키워드',
 '제출년도_2016',
 '제출년도_2017',
 '제출년도_2018',
 '제출년도_2019',
 '사업_부처명_경찰청',
 '사업_부처명_과학기술정보통신부',
 '사업_부처명_교육부',
 '사업_부처명_국민안전처',
 '사업_부처명_국토교통부',
 '사업_부처명_기상청',
 '사업_부처명_기획재정부',
 '사업_부처명_농림축산식품부',
 '사업_부처명_농촌진흥청',
 '사업_부처명_다부처',
 '사업_부처명_문화재청',
 '사업_부처명_문화체육관광부',
 '사업_부처명_미래창조과학부',
 '사업_부처명_범부처 사업',
 '사업_부처명_보건복지부',
 '사업_부처명_산림청',
 '사업_부처명_산업통상자원부',
 '사업_부처명_소방청',
 '사업_부처명_식품의약품안전처',
 '사업_부처명_원자력안전위원회',
 '사업_부처명_중소기업청',
 '사업_부처명_중소벤처기업부',
 '사업_부처명_특허청',
 '사업_부처명_해양경찰청',
 '사업_부처명_해양수산부',
 '사업_부처명_행정안전부',
 '사업_부처명_행정자치부',
 '사업_부처명_환경부',
 '계속과제여부_계속',
 '계속과제여부_신규']

In [35]:
test_dum = test_dum[['제출년도_2016', '제출년도_2017', '제출년도_2018', '제출년도_2019',
                       '사업_부처명_경찰청', '사업_부처명_과학기술정보통신부', '사업_부처명_교육부', '사업_부처명_국민안전처',
                       '사업_부처명_국토교통부', '사업_부처명_기상청', '사업_부처명_기획재정부', '사업_부처명_농림축산식품부',
                       '사업_부처명_농촌진흥청', '사업_부처명_다부처', '사업_부처명_문화재청', '사업_부처명_문화체육관광부',
                       '사업_부처명_미래창조과학부', '사업_부처명_범부처 사업', '사업_부처명_보건복지부', '사업_부처명_산림청',
                       '사업_부처명_산업통상자원부', '사업_부처명_소방청', '사업_부처명_식품의약품안전처',
                       '사업_부처명_원자력안전위원회', '사업_부처명_중소기업청', '사업_부처명_중소벤처기업부', '사업_부처명_특허청',
                       '사업_부처명_해양경찰청', '사업_부처명_해양수산부', '사업_부처명_행정안전부', '사업_부처명_행정자치부',
                       '사업_부처명_환경부', '계속과제여부_계속', '계속과제여부_신규']]

In [36]:
test_dum.shape

(43576, 34)

In [37]:
test_dum

,제출년도_2016,제출년도_2017,제출년도_2018,제출년도_2019,사업_부처명_경찰청,사업_부처명_과학기술정보통신부,사업_부처명_교육부,사업_부처명_국민안전처,사업_부처명_국토교통부,사업_부처명_기상청,...,사업_부처명_중소기업청,사업_부처명_중소벤처기업부,사업_부처명_특허청,사업_부처명_해양경찰청,사업_부처명_해양수산부,사업_부처명_행정안전부,사업_부처명_행정자치부,사업_부처명_환경부,계속과제여부_계속,계속과제여부_신규
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
4,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43571,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
43572,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
43573,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
43574,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [38]:
test_dum_csr_mat = csr_matrix(test_dum)

In [39]:
X_test = sp.hstack((test_title_token_vec, test_dum_csr_mat))

# Model Load

In [40]:
#test_clf = lgb.Booster(model_file='LGBM_v7.txt')

In [41]:
forest_pred1 = forest.predict(X_test)
forest_pred1

array([0, 0, 0, ..., 2, 0, 0], dtype=int64)

In [ ]:
#a = pd.DataFrame(lgb_pred1)

In [ ]:
#a.to_csv('./LGBM_v7_proba.csv')

lgb_pred2 = pd.DataFrame(columns=['INFO'])
lgb_pred2

lgb_pred2 = pd.DataFrame({'INFO' : lgb_pred1})
lgb_pred2

for idx, pred in tqdm(enumerate(lgb_pred2['INFO'])):
    if pred >= 0.5:
        lgb_pred2['INFO'].iloc[idx] = 1
    else:  
        lgb_pred2['INFO'].iloc[idx] = 0

# Submission

In [42]:
sample_submission = pd.read_csv('./submission/sample_submission.csv')

In [43]:
sample_submission['label'] = forest_pred1

In [ ]:
#sample_submission.drop(['label'], axis = 'columns', inplace = True)

In [ ]:
#sample_submission['label'] = lgb_pred2['INFO']

In [44]:
sample_submission.to_csv("./submission/RandomForest_{}.csv".format(version), index=False)

In [45]:
sample_submission['label'].describe()

count    43576.000000
mean         2.136176
std          7.302945
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         45.000000
Name: label, dtype: float64